In [1]:
import gym
import tensorflow as tf
import retro

import numpy as np
import random as r

from skimage import transform # to preprocess game frames
from skimage.color import rgb2gray # to convert game frames into grayscale

import matplotlib.pyplot as plt

from collections import deque # to create ordered collections of frames

import warnings # ignore warning messages from skiimage during training

warnings.filterwarnings("ignore")

In [3]:
env = gym.make('SpaceInvaders-v0') # make env

print("Frame Size: ", env.observation_space)
print("Action Size: ", env.action_space.n)

env.render()

Frame Size:  Box(210, 160, 3)
Action Size:  6


True